In [ ]:
%load_ext autoreload

In [ ]:
%autoreload
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from tianmoucv.data import TianmoucDataReader_pcie
import matplotlib.pyplot as plt


dataset_top1 = "/data/taoyi/dataset/Lyncam/expdata/zhuanpan/"
dataset_top2 = "/data/taoyi/dataset/Lyncam/driving_dataset/20230226"
datasetList = [dataset_top1,dataset_top2]

dataset = TianmoucDataReader_pcie(datasetList)

In [ ]:
%autoreload
import time
from tianmoucv.isp import lyncam_raw_comp,demosaicing_npy,fourdirection2xy
from tianmoucv.features.diff import HarrisCorner,align_images,sift,hog,feature_matching,mini_l2_cost_matching
import cv2

for sampleid in range(len(dataset)):
    if sampleid<1000:
        continue
    else:
        ###############################
        sample = dataset[sampleid]
        F1 = sample['F0']
        tsdiff = sample['rawDiff']/255.0
        threshed_tsdiff = tsdiff[:,0,...].permute(1,2,0)
        SD = threshed_tsdiff[...,1:]
        TD = threshed_tsdiff[...,0]
        Ix1,Iy1= fourdirection2xy(SD)
        Ix1 = torch.tensor(Ix1).float()
        Iy1 = torch.tensor(Iy1).float()

        sample = dataset[sampleid+33]
        F2 = sample['F0']
        tsdiff = sample['rawDiff']/255.0
        threshed_tsdiff = tsdiff[:,0,...].permute(1,2,0)
        SD = threshed_tsdiff[...,1:]
        TD = threshed_tsdiff[...,0]
        Ix2,Iy2= fourdirection2xy(SD)
        Ix2 = torch.tensor(Ix2).float()
        Iy2 = torch.tensor(Iy2).float()
        
        featureList1 = []
        kp1=[]
        featureList2 = []
        kp2=[]
        ###############################
        #计算两张图对应Harris角点检测
        startT = time.time()
        idmap,R = HarrisCorner(Ix1,Iy1,k=0.05,th=0.15)
        endT = time.time()
        idmap2,R = HarrisCorner(Ix2,Iy2,k=0.05,th=0.15)
        print('corner detect cost:',endT-startT)
        
        canvas = np.zeros([320,1280,3])
        canvas[:,:640,:] = F1
        canvas[:,640:,:] = F2
        #(step1)第一张图的feature list
        for i in range(idmap.shape[0]):
            for j in range(idmap.shape[1]):
                if idmap[i,j]>0:
                    cv2.circle(canvas,(j*4,i*2),4,(0,0,255))
                    kp1.append([i,j])
        #(step2)第二张图的feature list
        for i in range(idmap2.shape[0]):
            for j in range(idmap2.shape[1]):
                if idmap2[i,j]>0:
                    cv2.circle(canvas,(j*4+640,i*2),4,(255,0,0))
                    kp2.append([i,j])
                    
        #(step3)计算两张图对应fp list的特征描述子
        startT = time.time()
        kp1,featureList1 = sift(Ix1,Iy1,kp1)
        endT = time.time()
        print('feature extract cost:',endT-startT)
        kp2,featureList2 = sift(Ix2,Iy2,kp2) 
        print('good kp1/2:',len(kp1),len(kp2))
        print(kp1)
        #mapping to rgb coordinate
        
        kp1 = [ (p[0]*2,p[1]*4) for p in kp1 ]
        kp2 = [ (p[0]*2,p[1]*4) for p in kp2 ]
    
        fl1 = torch.stack(featureList1,dim=0).view(len(kp1),-1)
        fl2 = torch.stack(featureList2,dim=0).view(len(kp2),-1)
        
        matches = feature_matching(fl1, fl2, ratio=0.8)
        print('matched kp:',len(matches))
        
        
        canvas_wp, H = align_images(F1.copy(),kp1, kp2, matches, canvas) 

        plt.figure(figsize=(18,6))
        plt.imshow(canvas)
        plt.show()
        plt.figure(figsize=(18,6))
        plt.subplot(1,2,1)  
        plt.imshow(canvas_wp)
        plt.subplot(1,2,2)  
        plt.imshow(F2-canvas_wp)
        plt.show()
        
        break